In [ ]:
from difflib import SequenceMatcher
from geopy.distance import geodesic
import pandas as pd

In [ ]:
# https://en.wikipedia.org/wiki/List_of_United_States_cities_by_population
cities = pd.read_csv("cities.csv", index_col=[0, 1])

index = pd.DataFrame(index=cities.index)

In [ ]:
biking = pd.read_csv("biking/biking.csv", index_col=0)

index["biking"] = None

for i, row in cities.iterrows():
    city_state = "{}, {}".format(i[0], i[1])

    matches = biking["city"].apply(lambda x: SequenceMatcher(
        None, x, city_state).ratio()).sort_values(ascending=False)

    df = pd.DataFrame(biking.loc[matches.index, "city"])
    df["match"] = matches
    df.reset_index(inplace=True)

    if len(df) > 0:
        if df.loc[0, "match"] > 0.75:
            index.loc[i, "biking"] = df.loc[0, "Geography"]

    # if index.loc[city, "biking"] is None:
    #     print("{} not found".format(city_state))
    #     print(df.iloc[:5])

In [ ]:
housing = pd.read_csv("housing/housing.csv", index_col=0)

index["housing"] = 0

for i, row in cities.iterrows():
    matches = housing["city"].apply(lambda x: SequenceMatcher(
        None, x, i[0]).ratio()).sort_values(ascending=False)

    df = housing.loc[matches.index, ["city", "state"]]
    df.drop(df[df["state"] != row["abbrev"]].index, inplace=True)
    df["match"] = matches
    df.reset_index(inplace=True)

    if len(df) > 0:
        if df.loc[0, "match"] > 0.75:
            index.loc[i, "housing"] = df.loc[0, "RegionID"]

index.head()

In [ ]:
weather = pd.read_csv("weather/weather.csv", index_col=0)

index["weather"] = None

for i, row in cities.iterrows():
    df = weather.apply(lambda x: geodesic(
        (x["latitude"], x["longitude"]), (row["latitude"], row["longitude"])).miles, axis=1)
    df.sort_values(inplace=True)

    if df.iloc[0] < 15:
        index.loc[i, "weather"] = df.index[0]

    if index.loc[i, "weather"] is None:
        print("{}, {} not found".format(i[0], i[1]))
        print(df.iloc[:5])

In [ ]:
print(index.to_string())
index.dropna().to_csv("index.csv")